# Agents

Agents use an LLM to determine which actions to perform and in what order. An action can be either using a tool and observing its output or returning it to the user. To use an agent, in addition to the concept of an LLM, it is important to understand a new concept and that of a "tool".

# Tools

Tools are functions that agents can use to interact with the world. These tools can be common utilities (e.g. search), other chains, or even other agents

In [9]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
from langchain.agents import load_tools
from langchain_openai import ChatOpenAI
from langchain.agents import AgentType

llm = ChatOpenAI(model="gpt-4o-mini")

tool_names = ["llm-math"]
tools = load_tools(tool_names, llm=llm)
tools

[Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question

In [10]:
from langchain.agents import initialize_agent

agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True,
                         max_iterations=3)

Lets have a look at the ReAct template

In [12]:
agent.invoke(input="How many members does the Manchester United Team have?")
agent.invoke(input="What is 100 devided by 25?")



> Entering new AgentExecutor chain...
I should find out the current number of members on the Manchester United football team. A standard football team has 11 starting players, but there are also substitutes. Typically, a professional team has a full squad of around 25 to 30 players. I will look for the specific number that Manchester United has. 

Action: None yet as I don't have exact data.

Final Answer: As of my last update, a typical squad would consist of around 25 players, though specific numbers may vary slightly each season.

> Finished chain.


> Entering new AgentExecutor chain...
I need to perform the division of 100 by 25 to find the result.  
Action: Calculator  
Action Input: 100 / 25  
Observation: Answer: 4.0
Thought:I now know the final answer.  
Final Answer: 4.0

> Finished chain.


{'input': 'What is 100 devided by 25?', 'output': '4.0'}

# Custom Tools

You can also create your own tools by creating a class that inherits from BaseTool.

In [ ]:
import pickle

with open("vectorstore.pkl","rb") as f:
    vectorstore = pickle.load(f)

EOFError: Ran out of input

In [6]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS

embeddings = OpenAIEmbeddings()

vectorstore = FAISS.load_local("index", embeddings, allow_dangerous_deserialization=True)

RuntimeError: Error in __cdecl faiss::FileIOReader::FileIOReader(const char *) at D:\a\faiss-wheels\faiss-wheels\faiss\faiss\impl\io.cpp:68: Error: 'f' failed: could not open index\index.faiss for reading: No such file or directory

In [ ]:
from typing import Optional
from langchain.tools import BaseTool
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun

class CustomSearchTool(BaseTool):
    name: str = "restaurant search"
    description: str = "useful for when you need to answer questions about our restaurant"

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        store = vectorstore.as_retriever()
        docs = store.get_relevant_documents(query)
        text_list = [doc.page_content for doc in docs]
        return "\n".join(text_list)

    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

In [ ]:
from langchain.agents import AgentType

tools = [CustomSearchTool()]
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, max_iterations=3)

In [ ]:
agent.invoke(input="Who are the members of the A-Team?")


In [ ]:
agent.invoke(input="When does the restaurant open?")